In [ ]:
#| default_exp trig

# Useful trigonometry (trig)

> The trigonometric functions used to convert between relative and true direction and speed

These functions form the basis of the functions used in Appendix E of ITTC. This module is the trigonometry the underlying the appendix E. However, in general for seatrials work, you can use the wind evaluation module directly.

All the functions in this module can take arrays and perform vectorised operations.

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
import numpy as np
import pandas as pd
from fastcore.test import *

## Component functions

The first two functions in this  section are helper functions used to create the later functions.
These functions are

- $x\,\text{cos}(\theta)$
- $x\,\text{sin}(\theta)$

In [ ]:
#| export
def opposite_magnitude_fn(magnitude:float, #The true speed 
                             angle:float, #The angle in radians
                            ) -> int: #The vertical component of the magnitude
    "Product of sin and magnitude"
    
    x = magnitude * np.sin(angle)
    
    return x

In the example below a ship travelling due south (90$^\circ$ or $\frac{\pi}{2}$) at 20 knots, as a result the vertical component of its speed is $-20$.

In [ ]:
test_eq(opposite_magnitude_fn(20, np.pi/2), 20)

In [ ]:
#| export
def adjacent_magnitude_fn(magnitude:float, # The true speed
                             angle:float, # The Ange in radians
                             ) -> int: #The adjacent component of the magnitude
    
    "Product of cos and magnitude"
    
    x = magnitude * np.cos(angle)
    
    return x

Using the same example as the previous example we see that the resultant magnitude is 0

In [ ]:
test_eq(round(adjacent_magnitude_fn(20, np.pi/2), 5), 0)

# Combining vectors

It is often useful to combine two different vectors together e.g. ocean and tidal currents. This functions does the job

$$c \; \text{cos}(\gamma) = a \; \text{cos}(\alpha) + b \; \text{cos}(\beta), $$
$$c \; \text{sin}(\gamma) = a \; \text{sin}(\alpha) + b \; \text{sin}(\beta), $$

$$c = \sqrt{(c \; \text{cos}(\gamma)^2 + c \; \text{sin}(\gamma)^2)} $$

$$\gamma = \text{arctan2} \left( \frac{c \; \text{sin}(\gamma)}{c \; \text{cos}(\gamma)} \right)$$

The function takes the magnitude and angle of two vectors and outputs the magnitude and angle of the resultant vector.




In [ ]:
#| export
def combine_vectors(a:float, # magnitude of vector a
                    b:float,  #magnitude of vector b
                    alpha:float, #angle of vector a
                    beta:float #angle of vector b
                    ) -> float: # the magnitude and anngle of the new vector

    "Combine two 2-dimensional vectors into a new vector"
    #Temproary vectors which are removed when the function completes
    adj = a*np.cos(alpha) +  b*np.cos(beta)
    opp = a*np.sin(alpha) +  b*np.sin(beta)
    magnitude = np.sqrt(adj**2 + opp**2)

    #some values will be negative, is this a problem?
    gamma = np.arctan2(opp, adj)

    return magnitude, gamma

In [ ]:
#| hide

#I need some test here

## The Law of cosines

The law of cosines is used find the magnitude of a relative to b for a given angle between them.
In shipping this is used to find the relative windspeed from the true windspeed, relative current from true current, etc.

In [ ]:
#| export
def law_of_cosines(a:float, # side a which is along the x-axis
                   b:float, #side b makes the angle $\theta$ with side a
                   theta:float  #the angle in radians opposite side c
                  ) -> float: #The magnitude of b relative to a
    
    "Finds the length of side c using the angle theta opposite c and the length of the other two sides"
    
    adjacent_component = a - adjacent_magnitude_fn(b, theta)
    opposite_component = - opposite_magnitude_fn(b, theta)
    
    return np.sqrt(adjacent_component**2 + opposite_component**2)

In a right angled triangle the law of cosines simplifies to the pythagoras theorem

In [ ]:
test_eq(law_of_cosines(3,4, np.pi/2), 5)

## Finding an unknown angle of a triangle

Consider a triangle where the length of two sides are known and 1 angle is known where that angle is opposite the unknown side. Either of the unknown angles can be found by first using the single known angle and 1 of the known sides to create a right angle triangle, where the unknown side is the hypontenuse. From that using arctan the required angle can be found

$\text{tan} (\gamma) = \frac{a \, \text{sin}(\alpha))}{b + a \,\text{cos}(\alpha)}$

This means we can obtain the value of $\gamma$ using arctan


In [ ]:
#| export
def find_gamma_fn(a:int, #magnitude of a 
                  b:int, #magnitude of b
                  alpha:int, # the angle between b and a in radians
                  constrain_to_positive:bool = False #Should the function return a value between 0 and 2 pi
                 ) -> int:   #the angle in radians between a and the relative magnitude of b
    
        adjacent_component = b + adjacent_magnitude_fn(a, alpha)
        opposite_component = opposite_magnitude_fn(a, alpha)
        
        gamma = np.arctan2(opposite_component, adjacent_component)
        
        #prevents negative angles if constrain to positive is true
        #using this method instead of an if statement means the function can perform vectorised operations
        gamma = gamma + 2*np.pi*(gamma<0)*constrain_to_positive
        
        return gamma

Providing different values that are equally either side of $90^\circ$ produces values that are negative

In [ ]:
find_gamma_fn(np.array([3,3]), np.array([4, 4]), np.array([np.pi*0.5, np.pi*1.5]), constrain_to_positive = False)

array([ 0.64350111, -0.64350111])

In [ ]:
#| hide

#These tests just confirm that cosntrina to positive is working correctly
test_close (find_gamma_fn(3,4,np.pi*1.5), -0.64350, eps=1e-03)
test_close (find_gamma_fn(3,4,np.pi*1.5, constrain_to_positive = True), 5.6396, eps=1e-03)

By setting "constrain_to_positive" to true the function ensures that the angle stays the same but is flipped to be positive.

In [ ]:
find_gamma_fn([3,3], [4, 4], [np.pi*0.5, np.pi*1.5], constrain_to_positive = True)

array([0.64350111, 5.6396842 ])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()